In [1]:
dataPath = './Dataset/'
alldat = dataPath + 'CSVFiles-Legavy/'
allImg = dataPath + 'images/'

In [2]:
import pandas as pds
import numpy as np
import os
import pickle
from keras.utils import to_categorical

Using TensorFlow backend.


In [5]:
totalTypes = ['train', 'test', 'val']
csvFiles = [alldat + ('{}_all.csv'.format(x)) for x in totalTypes]
csvFiles

In [51]:
# for later

# flatList = [item for sublist in fullListInd for item in sublist]

# uniqueVocab.append('<start>')
# uniqueVocab.append('<end>')

# len(uniqueVocab)



In [7]:
# pandas dataframe 
dfA = pds.read_csv(csvFiles[0])
dfB = pds.read_csv(csvFiles[1])
dfC = pds.read_csv(csvFiles[2])

frames = [dfA, dfB, dfC]
df = pds.concat(frames, sort=False)

In [9]:
df.head()

,image_id,image_url,questions
0,90fbd03d-8de1-45e8-bd57-26419ac2fc31,http://www.davespianolessons.com/images/P13Z.jpg,Is the child a prodigy?---How long was the pra...
1,1236e320-0e85-42c5-9717-f1afffe3e41e,http://images5.fanpop.com/image/photos/2690000...,Why is Anne so surprised?---Why is she excited...
2,6e0b9faa-ad41-449b-9c3d-ff570e1568cd,http://rack.3.mshcdn.com/media/ZgkyMDEzLzA0LzI...,Why is this man in handcuffs?---Why is this ma...
3,3415bdf8-5f4b-4c7f-9522-7b510c359e83,http://3.bp.blogspot.com/-ab0Cd4ZbYTw/Thhb-MBq...,Are those all dairy cows?---Why are these chil...
4,3dc18299-6d97-4e0d-9e43-408f95b690f8,http://playerstrust.org/wp-content/uploads/201...,What kind of team is this?---What does the act...


In [53]:
picQueList = []
dictPic = dict()
wordList = set()

for i,row in df.iterrows():
    pictureID = row['image_id']
    questions = row['questions']
    questions = questions.split('---')
    
    for question in questions:
        
        if "?" in question:
            question = question[:question.index("?")]
            
        question = question.lower()
        question = "<start> " + question + " <end>"
        wordList.update(question.split())
        value = (pictureID, question)
        picQueList.append(value)

In [54]:
wordIndexMap = {val:index for index, val in enumerate(sorted(wordList))}

In [55]:
print (len(wordIndexMap))
print (wordIndexMap['<start>'])
print (wordIndexMap['<end>'])

10326
118
117


In [56]:
trainingLinkData = []
trainingXData = []
trainingYData = []
trainingQueNumber = []

for (link, qst) in picQueList:
    qst = qst.split()
    Xdat = []
    Ydat = []
    
    for i,word in enumerate(qst):
        maxInd = len(qst) - 2
        
        if i <= maxInd: 
            Xdat.append(wordIndexMap[word])
            Ydat.append(wordIndexMap[qst[i+1]])
        else: 
            continue
            
    trainingLinkData.append(link)
    trainingXData.append(Xdat)
    trainingYData.append(Ydat)

In [57]:
# filter out the values

filterTrainLinkData = []
filterTrainXData = []
filterTrainYData = []

filterTrainXLen = []
all_filenames = set(fn.rpartition('.')[0] for fn in os.listdir(allImg))

for image_id,token_in,token_out in zip(trainingLinkData, trainingXData, trainingYData):
    if image_id in all_filenames:
        filterTrainLinkData.append(image_id)
        filterTrainXData.append(token_in)
        filterTrainYData.append(token_out)

In [58]:
# find longest 

filterTrainXLen = [len(x) for x in filterTrainXData]
longestLen = (max(filterTrainXLen))
print (longestLen)

20


In [59]:
finalLink = []
finalX = []
finalY = []

for image_id,token_in,token_out in zip(filterTrainLinkData, filterTrainXData, filterTrainYData):
    limit = len(token_in) - 1
    
    for index, value in enumerate(token_in):
        new = token_in[:index + 1]
        finalLink.append(image_id)
        finalX.append(new)
        finalY.append(token_out[index])    


In [60]:
newFinalX = []

for val in finalX:
    padd = longestLen - len(val)
    padding = np.repeat(wordIndexMap['<end>'],padd).tolist()
    newFinalX.append((val + padding))


In [63]:
# convert to a dataframe here 

dataset = {'image_id':finalLink, 'token_in':newFinalX, 'token_out':finalY}

dataset_df = pds.DataFrame(dataset)
dataset_df.to_csv('full_dataset_indexes.csv', header=True, index=False)


In [41]:
dataset_df

,image_id,token_in,token_out
0,90fbd03d-8de1-45e8-bd57-26419ac2fc31,"[118, 117, 117, 117, 117, 117, 117, 117, 117, ...",4821
1,90fbd03d-8de1-45e8-bd57-26419ac2fc31,"[118, 4821, 117, 117, 117, 117, 117, 117, 117,...",9213
2,90fbd03d-8de1-45e8-bd57-26419ac2fc31,"[118, 4821, 9213, 117, 117, 117, 117, 117, 117...",1726
3,90fbd03d-8de1-45e8-bd57-26419ac2fc31,"[118, 4821, 9213, 1726, 117, 117, 117, 117, 11...",119
4,90fbd03d-8de1-45e8-bd57-26419ac2fc31,"[118, 4821, 9213, 1726, 119, 117, 117, 117, 11...",7043
5,90fbd03d-8de1-45e8-bd57-26419ac2fc31,"[118, 117, 117, 117, 117, 117, 117, 117, 117, ...",4535
6,90fbd03d-8de1-45e8-bd57-26419ac2fc31,"[118, 4535, 117, 117, 117, 117, 117, 117, 117,...",5372
7,90fbd03d-8de1-45e8-bd57-26419ac2fc31,"[118, 4535, 5372, 117, 117, 117, 117, 117, 117...",9945
8,90fbd03d-8de1-45e8-bd57-26419ac2fc31,"[118, 4535, 5372, 9945, 117, 117, 117, 117, 11...",9213
9,90fbd03d-8de1-45e8-bd57-26419ac2fc31,"[118, 4535, 5372, 9945, 9213, 117, 117, 117, 1...",6934


In [65]:
pickle.dump(wordIndexMap, open( "word2IndexDict.p", "wb" ) ) # we already have this saved

In [29]:
dataset_df.to_csv("finalDataset-Agg.csv",index=False)